In [2]:
! git clone https://github.com/zabir-nabil/drug-drug-interaction.git

Cloning into 'drug-drug-interaction'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 32 (delta 2), reused 28 (delta 2), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [3]:
! pip install transformers

     |████████████████████████████████| 2.8 MB 5.4 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 895 kB 37.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.0 MB/s 
     |████████████████████████████████| 636 kB 40.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
% cd drug-drug-interaction/bert_code/

/content/drug-drug-interaction/bert_code


In [5]:
! python main.py --num_train_epochs 1 --do_train

09/21/2021 16:34:06 - INFO - filelock -   Lock 139658255457488 acquired on /root/.cache/huggingface/transformers/f56489506305d1a3dbdde9494ec55bd66571a067b3eb113baf104ce700513f41.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock
Downloading: 100% 213k/213k [00:00<00:00, 2.06MB/s]
09/21/2021 16:34:07 - INFO - filelock -   Lock 139658255457488 released on /root/.cache/huggingface/transformers/f56489506305d1a3dbdde9494ec55bd66571a067b3eb113baf104ce700513f41.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock
09/21/2021 16:34:07 - INFO - filelock -   Lock 139658255457424 acquired on /root/.cache/huggingface/transformers/ed9bbe6348755db01a526f9467c73a8c8f55a43191f892374c9ed386b4525997.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d.lock
Downloading: 100% 112/112 [00:00<00:00, 129kB/s]
09/21/2021 16:34:07 - INFO - filelock -   Lock 139658255457424 released on /root/.cache/huggingface/transformers/ed9bbe6348755db01a526f9467c73a8c8f55a43191

In [3]:
import argparse

from trainer import Trainer
from utils import init_logger, load_tokenizer
from data_loader import load_and_cache_examples

In [4]:
from types import SimpleNamespace

args =    {
    "pretrained_model_name" : "monologg/biobert_v1.1_pubmed",
    "do_lower_case" : True,
    "task" : "ddi",
    "data_dir" : "./data",
    "model_dir" : "./model",
    "train_file" : "train.tsv",
    "test_file" : "test.tsv",
    "label_file" : "label.txt",
    "seed" : 42,
    "batch_size" : 10,
    "max_seq_len" : 350,
    "learning_rate" : 2e-5,
    "num_train_epochs" : 40,
    "weight_decay" : 0.0,
    "gradient_accumulation_steps" : 1,
    "adam_epsilon" : 1e-8,
    "max_grad_norm" : 1.0,
    "max_steps" : -1,
    "warmup_steps" : 0,
    "dropout_rate" : 0.1,
    "logging_steps" : 400,
    "save_steps" : 400,
    "do_train" : True,
    "do_eval" : True,
    "no_cuda" : False

    }
args = SimpleNamespace(**args)

In [6]:
print(args.pretrained_model_name)

init_logger()
tokenizer = load_tokenizer(args)
# test_dataset = load_and_cache_examples(args, tokenizer, mode="test")

monologg/biobert_v1.1_pubmed


In [7]:
#for t in test_dataset:
#  print(t)
#  break

In [8]:
import os
import logging
from tqdm import tqdm, trange

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertConfig, AdamW, get_linear_schedule_with_warmup

from model import RBERT
from utils import set_seed, compute_metrics, get_label

In [9]:
#eval_sampler = SequentialSampler(test_dataset)
#eval_dataloader = DataLoader(test_dataset, sampler=eval_sampler, batch_size=1)

In [10]:
preds = None
out_label_ids = None

bert_config = BertConfig.from_pretrained(args.pretrained_model_name, num_labels= 5, finetuning_task=args.task)
model = RBERT(bert_config, args)
model.eval()
device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
model.to(device)

Some weights of the model checkpoint at monologg/biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      

In [14]:
for batch in tqdm(eval_dataloader, desc="Evaluating"):
    print("-------------------------------------")
    print(batch)
    batch = tuple(t.to(device) for t in batch)
    
    with torch.no_grad():
        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'token_type_ids': batch[2],
                  'labels': batch[3],
                  'e1_mask': batch[4],
                  'e2_mask': batch[5]}
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]
        print(inputs)
        print(outputs)
        break

In [11]:
from data_loader import *
processors = {
    "ddi": DDIProcessor
}
processor = processors[args.task](args)

In [27]:
examples = processor.get_examples("test")
features = []

max_seq_len = args.max_seq_len
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token_id = tokenizer.pad_token_id

cls_token_segment_id=0
pad_token_segment_id=0
sequence_a_segment_id=0
mask_padding_with_zero=True

labels_all = ["negative", "mechanism", "effect", "advise", "int"]

results_test = open("test_results.txt", "w")
dict_results = {
    
}

for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    e11_p = tokens_a.index("<e1>")  # the start position of entity1
    e12_p = tokens_a.index("</e1>")  # the end position of entity1
    e21_p = tokens_a.index("<e2>")  # the start position of entity2
    e22_p = tokens_a.index("</e2>")  # the end position of entity2

    # Replace the token
    tokens_a[e11_p] = "$"
    tokens_a[e12_p] = "$"
    tokens_a[e21_p] = "#"
    tokens_a[e22_p] = "#"

    # Add 1 because of the [CLS] token
    e11_p += 1
    e12_p += 1
    e21_p += 1
    e22_p += 1

    # Account for [CLS] and [SEP] with "- 2" and with "- 3" for RoBERTa.
    special_tokens_count = 2
    if len(tokens_a) > max_seq_len - special_tokens_count:
        tokens_a = tokens_a[:(max_seq_len - special_tokens_count)]

    tokens = tokens_a
    tokens += [sep_token]

    token_type_ids = [sequence_a_segment_id] * len(tokens)

    tokens = [cls_token] + tokens
    token_type_ids = [cls_token_segment_id] + token_type_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    attention_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_len - len(input_ids)
    input_ids = input_ids + ([pad_token_id] * padding_length)
    attention_mask = attention_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
    token_type_ids = token_type_ids + ([pad_token_segment_id] * padding_length)

    # e1 mask, e2 mask
    e1_mask = [0] * len(attention_mask)
    e2_mask = [0] * len(attention_mask)

    for i in range(e11_p, e12_p + 1):
        e1_mask[i] = 1
    for i in range(e21_p, e22_p + 1):
        e2_mask[i] = 1

    assert len(input_ids) == max_seq_len, "Error with input length {} vs {}".format(len(input_ids), max_seq_len)
    assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
    assert len(token_type_ids) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_ids), max_seq_len)

    label_id = int(example.label)

    if ex_index == 0:
        print(input_ids)
        print(label_id)
        print(attention_mask)
        print(e1_mask)
        print(e2_mask)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)
    all_attention_mask = torch.tensor([attention_mask], dtype=torch.long).to(device)
    all_token_type_ids = torch.tensor([token_type_ids], dtype=torch.long).to(device)
    all_e1_mask = torch.tensor([e1_mask], dtype=torch.long).to(device)  # add e1 mask
    all_e2_mask = torch.tensor([e2_mask], dtype=torch.long).to(device)  # add e2 mask
    all_label_ids = torch.tensor([label_id], dtype=torch.long).to(device)

    with torch.no_grad():
        inputs = {'input_ids': all_input_ids,
                  'attention_mask': all_attention_mask,
                  'token_type_ids': all_token_type_ids,
                  'labels': all_label_ids,
                  'e1_mask': all_e1_mask,
                  'e2_mask': all_e2_mask}
        outputs = model(**inputs)
        # print(labels_all[torch.argmax(outputs[1])])

        print("input_text: ", example.text_a)
        print("original_label: ", labels_all[all_label_ids])
        print("prediction: ", labels_all[torch.argmax(outputs[1])])

        results_test.write(f"{example.text_a}|{all_label_ids}|{labels_all[torch.argmax(outputs[1])]}")


09/21/2021 18:08:34 - INFO - data_loader -   LOOKING AT ./data/test.tsv
09/21/2021 18:08:34 - INFO - data_loader -   ['negative', 'In the present study, we tested whether the highly specific metabotropic glutamate receptor # antagonist, <e1> 3-[(2-methyl-1,3-thiazol-4-yl)_ethynyl]_pyridine </e1> , reduces dopaminergic and noradrenergic neuronal loss in monkeys rendered parkinsonian by chronic treatment with low doses of <e2> 1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine </e2> .']
09/21/2021 18:08:34 - INFO - data_loader -   ['effect', 'Concurrent use of <e1> alcohol </e1> and other CNS_depression-producing_drugs may increase the CNS depressant effects of <e2> methyprylon </e2> or these other medications.']
09/21/2021 18:08:34 - INFO - data_loader -   ['negative', '<e1> TRENTAL </e1> has been used concurrently with antihypertensive_drugs , beta_blockers , <e2> digitalis </e2> , diuretics , antidiabetic_agents , and antiarrhythmics , without observed problems.']
09/21/2021 18:08:34 - INFO

[101, 1107, 1103, 1675, 2025, 117, 1195, 7289, 2480, 1103, 3023, 2747, 27154, 19937, 27098, 176, 25937, 7363, 1566, 10814, 108, 19173, 117, 109, 124, 118, 164, 113, 123, 118, 1899, 18873, 118, 122, 117, 124, 118, 24438, 1465, 6112, 1233, 118, 125, 118, 194, 1233, 114, 168, 3084, 7889, 3382, 1233, 166, 168, 185, 12577, 2386, 2042, 109, 117, 13822, 1202, 4163, 9685, 10805, 1596, 1105, 4040, 3556, 16717, 10805, 1596, 24928, 22497, 1348, 2445, 1107, 22413, 10029, 2493, 4935, 11357, 1179, 1118, 13306, 3252, 1114, 1822, 24429, 1104, 108, 122, 118, 1899, 18873, 118, 125, 118, 185, 10436, 7777, 118, 122, 117, 123, 117, 124, 117, 127, 118, 21359, 4487, 7889, 23632, 22753, 10132, 2042, 108, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: ignored

In [14]:
def str2intarr(a):
  return [ord(a_i) for a_i in a]

In [11]:
str2intarr('asa asda da,as dasd,as dasdasd 234 sfsdf')

[97,
 115,
 97,
 32,
 97,
 115,
 100,
 97,
 32,
 100,
 97,
 44,
 97,
 115,
 32,
 100,
 97,
 115,
 100,
 44,
 97,
 115,
 32,
 100,
 97,
 115,
 100,
 97,
 115,
 100,
 32,
 50,
 51,
 52,
 32,
 115,
 102,
 115,
 100,
 102]

In [12]:
def intarr2str(a):
  return "".join(chr(a_i) for a_i in a)

In [13]:
intarr2str([97,
 115,
 97,
 32,
 97,
 115,
 100,
 97,
 32,
 100,
 97,
 44,
 97,
 115,
 32,
 100,
 97,
 115,
 100,
 44,
 97,
 115,
 32,
 100,
 97,
 115,
 100,
 97,
 115,
 100,
 32,
 50,
 51,
 52,
 32,
 115,
 102,
 115,
 100,
 102])

'asa asda da,as dasd,as dasdasd 234 sfsdf'